In [320]:
# import pandas (library for dataframes)
import pandas as pd
pd.set_option('display.max_columns', None)
# import listdir (library to list directories)
from os import listdir
import numpy as np
from datetime import datetime, timedelta
import xlsxwriter
import inflect
import math

In [321]:
def breathing_calculations(file_name, file_suffix, file_location):

    # Check if file exists in file location
    file_exists = False
    for dir_file in listdir(file_location):
        if dir_file == file_name + file_suffix:
            file_exists = True
    
    if not file_exists:
        print("File: " + file_name + file_suffix + " does not exist in " + file_location)
        raise Exception("File: " + file_name + file_suffix + " does not exist in " + file_location)

    # Read in data from file into dataframe
    data = None
    column_names = ["row_number","cpu_date","cpu_time","site_time","period_time","Action","title",
                "description","Blank", "Breath_Num","first_beat_id","last_beat_id", "Ti","Te",
                "PIF","PEF", "TV", "EV","RT","MV", "f", "EIP", "EEP", "Penh", "EF50", "RH", 
                "Tbox", "Tbody", "Patm", "VCF", "AV", "Sr", "n"]
    with open(file_location + file_name + file_suffix, encoding='unicode_escape') as f:
        data = pd.read_csv(f, header=None, names=column_names, skip_blank_lines=True, na_filter=True, dtype='string', skipinitialspace=True, sep='\t')

    # row numbers when Action column is period-start
    period_start_rows = data[data["Action"] == "period-start"].index

    # start the data at the row of the first period-start Action
    first_period_start = int(period_start_rows[0])
    print(first_period_start)

    # row numbers when Action column is period-stop
    period_stop_rows = data[data["Action"] == "period-stop "].index

    # end the data at the row of the last period-stop Action
    last_period_stop = int(period_stop_rows[-1])
    print(last_period_stop)

    # change data to be from first period-start to last period-stop
    data = data.iloc[first_period_start:last_period_stop+1]

    # Drop unnecessary columns/rows
    data = data.drop(["first_beat_id","last_beat_id"], axis="columns")
    data = data.drop("Blank", axis="columns")
    data = data.drop(["row_number", "cpu_date", "cpu_time"], axis="columns")

    analyzer_row_numbers = data[data["Action"] == "analyzer-tuning"].index
    data = data.drop(analyzer_row_numbers)

    # convert necessary columns to float types
    num_cols = len(data.columns)
    list_cols_to_convert_to_float = list(range(6, num_cols))
    for i in list_cols_to_convert_to_float:
        data.iloc[:, i] = data.iloc[:, i].map(float, na_action='ignore')

    # Reset index to start from 0
    data = data.reset_index(drop=True)

    # Period start and period stop rows change after reset_index, so reassign variables
    period_start_rows = data[data["Action"] == "period-start"].index
    period_stop_rows = data[data["Action"] == "period-stop "].index

    # Number of Breaths per condition
    num_breaths_per_condition = period_stop_rows - period_start_rows - 1
    print(num_breaths_per_condition)

    # Get title condition for each period-start Action
    conditions = list(data[data["Action"] == "period-start"]["title"].values)
    print(conditions)

    # Replace Ramp if exists
    if "Hypoxia" in conditions:
        hypoxia_index = conditions.index("Hypoxia")
        # Change Ramp to Ramp Up After Hypoxia (if exists)
        if conditions[hypoxia_index + 1] == "Ramp":
            conditions[hypoxia_index + 1] = "Ramp Up"

        # Change Ramp to Ramp Down Before Hypoxia (if exists)
        if conditions[hypoxia_index - 1] == "Ramp":
            conditions[hypoxia_index - 1] = "Ramp Down"

    # Keep all conditions not equal (ne) to Ramp
    # If ramp is removed, be sure to remove ramp data as well
    # conditions = list(filter("Ramp".__ne__, conditions))
    # print(conditions)

    # Loop through number of conditions to average data for each condition
    averaged_data = pd.DataFrame()
    for i in range(len(conditions)):
        condition_start_row = period_start_rows[i] + 1
        condition_stop_row = period_stop_rows[i] - 1
        condition_rows_data = data.iloc[condition_start_row:condition_stop_row, 6:]
        filtered_TV = condition_rows_data[condition_rows_data["TV"] < 0.8]
        one_averaged_data = filtered_TV.mean()
        averaged_data = averaged_data.append(one_averaged_data, ignore_index=True)

    # Join Breaths Dataframe with Averages Dataframe
    breaths_dataframe = pd.DataFrame(num_breaths_per_condition, columns=["Breath_Count"])
    averaged_data = breaths_dataframe.join(averaged_data)

    # Differentiate Conditions using Dictionary to keep track of number of duplicates
    numbered_conditions = []
    number_dict = {}
    for condition in conditions:
        if condition in number_dict:
            number_dict[condition] += 1
            numbered_conditions.append(condition + " " + str(number_dict[condition]))
        else:
            number_dict[condition] = 1
            numbered_conditions.append(condition)

    # Join Conditions Dataframe with Averages Dataframe
    conditions_dataframe = pd.DataFrame(numbered_conditions, columns=["Conditions"])
    averaged_data = conditions_dataframe.join(averaged_data)

    # Condition reduction
    averaged_data = condition_reduction("Baseline", "Post Injection", "Base_PI_Red", averaged_data)

    # Add 5Min Bins
    recovery_5min_averaged_data = create_5min_bins(data, "Recovery", 5, averaged_data)
    postinjection_recovery_5min_averaged_data = create_5min_bins(data, "Post Injection", 5, recovery_5min_averaged_data)

    # Add the first 10min and first 15min of Post injection
    post_injection_first_10_averaged = create_own_bins(data, "Post Injection", 0, 10, postinjection_recovery_5min_averaged_data)
    post_injection_and_first_15_averaged = create_own_bins(data, "Post Injection", 0, 15, post_injection_first_10_averaged)

    # Condition reduction (baseline, post injection first 10), (baseline, post injection first 15)
    combined_data = condition_reduction("Baseline", "PI_0-10", "10_Reduct", post_injection_and_first_15_averaged)
    combined_data = condition_reduction("Baseline", "PI_0-15", "15_Reduct", combined_data)

    # Duplicate file_directory row information averaged_data index # of times to join together
    file_dir_row = file_dir_data[file_dir_data["FileName"] == file_name]
    file_dir_row = file_dir_row.append([file_dir_row]*(len(post_injection_and_first_15_averaged.index) - 1),ignore_index=True)

    # Join file_directory mouse info with averaged_data
    mouse_averaged_data = file_dir_row.join(combined_data)
    return mouse_averaged_data

In [322]:
def condition_mean(cond1, cond2, row_name, averaged_data):
    if cond1 in averaged_data["Conditions"].values and cond2 in averaged_data["Conditions"].values:
        cond1_data = averaged_data[averaged_data["Conditions"] == cond1].iloc[:, 1:].values[0]
        cond2_data = averaged_data[averaged_data["Conditions"] == cond2].iloc[:, 1:].values[0]
        mean = [(cond1_i + cond2_i) / 2 for cond1_i, cond2_i in zip(cond1_data, cond2_data)]
        mean.insert(0, row_name)
        averaged_data.loc[len(averaged_data.index)] = mean
        return averaged_data

In [323]:
def condition_reduction(cond1, cond2, row_name, averaged_data):
    if cond1 in averaged_data["Conditions"].values and cond2 in averaged_data["Conditions"].values:
        # Get values of condition 1 data and condition 2 data as lists
        cond1_data = averaged_data[averaged_data["Conditions"] == cond1].iloc[:, 1:].values[0]
        cond2_data = averaged_data[averaged_data["Conditions"] == cond2].iloc[:, 1:].values[0]
        reduction = list((np.subtract(cond2_data, cond1_data) / cond1_data) * 100)
        reduction.insert(0, row_name)
        averaged_data.loc[len(averaged_data.index)] = reduction
        return averaged_data

In [324]:
# data - cleaned data
# title - the title/condition 5 min bin
# averaged data - averaged analysis table
def create_5min_bins(data, title, frequency, averaged_data):
    title_index = data[data["title"] == title].index

    # Excludes the period-start, period-stop rows
    title_data_rows = data.iloc[title_index[0]+1:title_index[1], :]

    # Convert period time to allow math calculations
    title_period_time = title_data_rows["period_time"]
    lst_time = []
    for period_time in title_period_time:
        new_time = None
        try:
            new_time = datetime.strptime(period_time, '%M:%S.%f')
        except:
            new_time = datetime.strptime(period_time, '%H:%M:%S.%f')
        
        lst_time.append(new_time)

    title_data_rows.loc[:, "period_time"] = lst_time

    # Set index to allow groupby 5 min aggregations
    title_data_rows = title_data_rows.set_index("period_time")

    # Convert analysis columns to floats to allow calculations for averages
    title_cols = len(title_data_rows.columns)
    list_cols_to_convert_to_float = list(range(4, title_cols))
    for i in list_cols_to_convert_to_float:
        title_data_rows.iloc[:, i] = title_data_rows.iloc[:, i].map(float, na_action='ignore')

    # Groupby 5 minute intervals, count number of rows in each 5 minute interval to determine breath count
    title_breath_num_count = title_data_rows.groupby(pd.Grouper(freq=str(frequency) + 'T')).agg({"Breath_Num":  "count"})
    title_breath_num_count = title_breath_num_count.reset_index(drop=True)

    # Take mean of analysis columns by 5 minute intervals
    title_analysis_5m_means = title_data_rows.iloc[:, 5:].resample(str(frequency) + "T").mean()
    title_analysis_5m_means = title_analysis_5m_means.reset_index(drop=True)

    # Create condition names for each 5 min interval for title (e.g. PI_first5, PI_second5, etc.)
    title_name = None
    if title == "Post Injection":
        title_name = "PI"
    elif title == "Recovery":
        title_name = "REC"
    else:
        title_name = "Title"

    num_title_rows = len(title_analysis_5m_means.index)
    list_5min_title_condition = []
    inflect_engine = inflect.engine()
    for i in range(1, num_title_rows + 1):
        list_5min_title_condition.append(title_name + "_" + inflect_engine.number_to_words(inflect_engine.ordinal(i)) + str(frequency))
    
    title_condition_df = pd.DataFrame(list_5min_title_condition, columns=["Conditions"])
    
    # Combine condition with breath count and analysis averages dataframe
    condition_breath = title_condition_df.join(title_breath_num_count)
    condition_breath_analysis = condition_breath.join(title_analysis_5m_means)
    condition_breath_analysis = condition_breath_analysis.rename(columns={"Breath_Num": "Breath_Count"})

    # Combine new 5Min Bin Dataframe with other averaged data conditions
    averages_and_5MinAnalysis = averaged_data.append(condition_breath_analysis)
    return averages_and_5MinAnalysis

In [325]:
# data - cleaned data
# title - the title/condition 5 min bin
# averaged data - averaged analysis table
def create_own_bins(data, title, start, end, averaged_data):
    title_index = data[data["title"] == title].index

    # Excludes the period-start, period-stop rows
    title_data_rows = data.iloc[title_index[0]+1:title_index[1], :]

    # Convert period time to allow math calculations
    title_period_time = title_data_rows["period_time"]
    lst_time = []
    for period_time in title_period_time:
        new_time = None
        try:
            new_time = datetime.strptime(period_time, '%M:%S.%f')
        except:
            new_time = datetime.strptime(period_time, '%H:%M:%S.%f')
        
        lst_time.append(new_time)

    title_data_rows.loc[:, "period_time"] = lst_time

    rows_between_time = title_data_rows[(title_data_rows["period_time"].dt.minute > start) & (title_data_rows["period_time"].dt.minute < end)]

    num_breaths = [int(list(rows_between_time["Breath_Num"])[-1]) - int(list(rows_between_time["Breath_Num"])[0])]
    num_breaths_df = pd.DataFrame(num_breaths, columns=["Breath_Num"])

    # Convert analysis columns to floats to allow calculations for averages
    title_cols = len(title_data_rows.columns)
    list_cols_to_convert_to_float = list(range(6, title_cols))
    for i in list_cols_to_convert_to_float:
        title_data_rows.iloc[:, i] = title_data_rows.iloc[:, i].map(float, na_action='ignore')

    mean_analysis_df = title_data_rows.iloc[:, 6:].mean(axis=0)
    analysis_average_df = pd.DataFrame(mean_analysis_df.to_dict(), index=[0])

    # Create condition names for each 5 min interval for title (e.g. PI_first5, PI_second5, etc.)
    title_name = None
    if title == "Post Injection":
        title_name = "PI"
    elif title == "Recovery":
        title_name = "REC"
    else:
        title_name = "Title"
    
    lst_title_name = [title_name + "_" + str(start) + "-" + str(end)]

    title_condition_df = pd.DataFrame(lst_title_name, columns=["Conditions"])
    
    # Combine condition with breath count and analysis averages dataframe
    condition_breath = title_condition_df.join(num_breaths_df)
    condition_breath_analysis = condition_breath.join(analysis_average_df)
    condition_breath_analysis = condition_breath_analysis.rename(columns={"Breath_Num": "Breath_Count"})

    # # Combine new 5Min Bin Dataframe with other averaged data conditions
    averages_and_5MinAnalysis = averaged_data.append(condition_breath_analysis)
    return averages_and_5MinAnalysis

In [326]:
# open file directory and save data to dataframe
with open('inputs/injection_cd-1/___FILE DIRECTORY___.csv', encoding='unicode_escape') as f:
    file_dir_data = pd.read_csv(f, header=0, skip_blank_lines=True, na_filter=True, dtype='string', skipinitialspace=True, sep=',')
file_dir_data

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Include,Baseline_Filter_Less_Strict,Baseline_Filter_More_Strict,Experiment_Type,Anesthesia,Anesthesia_Dose,Flow_Threshold,Injection_Type,Injection_Success,Injection_Attempts,Injection1,Injection1_Stock_Conc,Injection1_Stock_Conc_Units,Injection1_Stock_Prep_Date,Injection1_Solvent,Injection1_Solvent_Percent,Injection1_Dosage,Injection1_Dosage_Units,Injection1_volume_ml,Injection2,Injection2_Stock_Conc,Injection2_Stock_Conc_Units,Injection2_Stock_Prep_Date,Injection2_Solvent,Injection2_Solvent_Percent,Injection2_Dosage,Injection2_Dosage_Units,Injection2_volume_ml,Protocol,Raw_Protocol,Notes,Notes for Nick
0,221112_chamber1_ajc7482_m02wh,1,1,TL + MC,2022-11-12,<NA>,ajc7482,m,wh,2,filler,filler,filler,filler,filler,34.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>
1,221112_chamber2_ajc7483_m20wh,2,1,TL + MC,2022-11-12,<NA>,ajc7483,m,wh,20,filler,filler,filler,filler,filler,43.3,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>
2,221112_chamber1_ajc7484_m03wh,1,1,TL + MC,2022-11-12,<NA>,ajc7484,m,wh,3,filler,filler,filler,filler,filler,33.2,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>
3,221112_chamber2_ajc7485_mwh,2,1,TL + MC,2022-11-12,<NA>,ajc7485,m,wh,0,filler,filler,filler,filler,filler,38.1,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>
4,221113_chamber1_ajc7478_m01wh,1,1,DA + MC,2022-11-13,<NA>,ajc7478,m,wh,1,filler,filler,filler,filler,filler,40.1,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>
5,221113_chamber2_ajc7479_m02wh,2,1,DA + MC,2022-11-13,<NA>,ajc7479,m,wh,2,filler,filler,filler,filler,filler,38.8,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>
6,221113_chamber1_ajc7480_m03wh,1,1,DA + MC,2022-11-13,<NA>,ajc7480,m,wh,3,filler,filler,filler,filler,filler,33.2,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>
7,221113_chamber2_ajc7481_m10wh,2,1,DA + MC,2022-11-13,<NA>,ajc7481,m,wh,10,filler,filler,filler,filler,filler,30.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>
8,221118_chamber1_ajc7484_m03wh,1,2,AC + MC,2022-11-18,<NA>,ajc7484,m,wh,3,filler,filler,filler,filler,filler,33.3,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>
9,221118_chamber2_ajc7483_m20wh,2,2,AC + MC,2022-11-18,<NA>,ajc7483,m,wh,20,filler,filler,filler,filler,filler,45.3,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post I

In [327]:
# Convert weight column to float for calculations
file_dir_data["Weight"] = file_dir_data["Weight"].map(float, na_action=None)

# Convert Date_Tested column to datetime for calculations
file_dir_data["Date_Tested"] = file_dir_data["Date_Tested"].map(lambda date: datetime.strptime(date, "%Y-%m-%d"))

# Convert DOB column to datetime for calculations
# file_dir_data["DOB"] = file_dir_data["DOB"].map(lambda date: datetime.datetime.strptime(date, "%Y-%m-%d"))

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [328]:
# Add Age column
# file_dir_data["Age"] = file_dir_data["Date_Tested"] - file_dir_data["DOB"]
file_dir_data["Age"] = fillna()

NameError: name 'fillna' is not defined

In [329]:
analysis_names = ["Conditions","Breath_Count", "Ti","Te","PIF","PEF", 
                "TV", "EV","RT","MV", "f", "EIP", "EEP", "Penh", "EF50", 
                "RH", "Tbox","Tbody", "Patm","VCF", "AV", "Sr","n"]

master_data_columns = list(file_dir_data.columns.values) + analysis_names
master_data_columns

['FileName',
 'Chamber',
 'Test_Number',
 'Tester',
 'Date_Tested',
 'DOB',
 'Mouse_ID',
 'Sex',
 'Coat_Color',
 'Earclip',
 'Mouse_Strain',
 'Gene',
 'Genotype',
 'Gene2',
 'Genotype2',
 'Weight',
 'Include',
 'Baseline_Filter_Less_Strict',
 'Baseline_Filter_More_Strict',
 'Experiment_Type',
 'Anesthesia',
 'Anesthesia_Dose',
 'Flow_Threshold',
 'Injection_Type',
 'Injection_Success',
 'Injection_Attempts',
 'Injection1',
 'Injection1_Stock_Conc',
 'Injection1_Stock_Conc_Units',
 'Injection1_Stock_Prep_Date',
 'Injection1_Solvent',
 'Injection1_Solvent_Percent',
 'Injection1_Dosage',
 'Injection1_Dosage_Units',
 'Injection1_volume_ml',
 'Injection2',
 'Injection2_Stock_Conc',
 'Injection2_Stock_Conc_Units',
 'Injection2_Stock_Prep_Date',
 'Injection2_Solvent',
 'Injection2_Solvent_Percent',
 'Injection2_Dosage',
 'Injection2_Dosage_Units',
 'Injection2_volume_ml',
 'Protocol',
 'Raw_Protocol',
 'Notes',
 'Notes for Nick',
 'Conditions',
 'Breath_Count',
 'Ti',
 'Te',
 'PIF',
 'PEF',
 

In [330]:
# Create empty master dataframe
master_data = pd.DataFrame(columns=master_data_columns)
master_data
# TODO: Should I do something with this?
# master_data$Date_Tested <- as.Date(master_data$Date_Tested,"%Y-%m-%d")
# master_data$DOB <- as.Date(master_data$DOB, "%m/%d/%y")

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Include,Baseline_Filter_Less_Strict,Baseline_Filter_More_Strict,Experiment_Type,Anesthesia,Anesthesia_Dose,Flow_Threshold,Injection_Type,Injection_Success,Injection_Attempts,Injection1,Injection1_Stock_Conc,Injection1_Stock_Conc_Units,Injection1_Stock_Prep_Date,Injection1_Solvent,Injection1_Solvent_Percent,Injection1_Dosage,Injection1_Dosage_Units,Injection1_volume_ml,Injection2,Injection2_Stock_Conc,Injection2_Stock_Conc_Units,Injection2_Stock_Prep_Date,Injection2_Solvent,Injection2_Solvent_Percent,Injection2_Dosage,Injection2_Dosage_Units,Injection2_volume_ml,Protocol,Raw_Protocol,Notes,Notes for Nick,Conditions,Breath_Count,Ti,Te,PIF,PEF,TV,EV,RT,MV,f,EIP,EEP,Penh,EF50,RH,Tbox,Tbody,Patm,VCF,AV,Sr,n


In [331]:
# Retrieve file list from file directory
file_list = file_dir_data["FileName"]
file_list

0     221112_chamber1_ajc7482_m02wh
1     221112_chamber2_ajc7483_m20wh
2     221112_chamber1_ajc7484_m03wh
3       221112_chamber2_ajc7485_mwh
4     221113_chamber1_ajc7478_m01wh
5     221113_chamber2_ajc7479_m02wh
6     221113_chamber1_ajc7480_m03wh
7     221113_chamber2_ajc7481_m10wh
8     221118_chamber1_ajc7484_m03wh
9     221118_chamber2_ajc7483_m20wh
10    221118_chamber1_ajc7485_m00wh
11    221118_chamber2_ajc7482_m02wh
12    221120_chamber1_ajc7481_m10wh
13    221120_chamber2_ajc7480_m03wh
14    221120_chamber1_ajc7479_m02wh
15    221120_chamber2_ajc7478_m01wh
16    221124_chamber1_ajc7482_m02wh
17    221124_chamber2_ajc7483_m20wh
18    221124_chamber1_ajc7484_m03wh
19    221124_chamber2_ajc7485_m00wh
20    221127_chamber1_ajc7478_m01wh
21    221127_chamber2_ajc7480_m03wh
22    221127_chamber1_ajc7481_m10wh
23    221127_chamber2_ajc7479_m02wh
24    221202_chamber1_ajc7482_m02wh
25    221202_chamber2_ajc7483_m20wh
26    221202_chamber1_ajc7484_m03wh
27    221202_chamber2_ajc748

In [332]:
# Apply breathing_calculations to each file
file_location = "inputs/injection_cd-1/"
file_suffix = ".rf_1.iox.txt"
for file_name in file_list:
    master_data = master_data.append(breathing_calculations(file_name, file_suffix, file_location))
master_data

211
8661
Int64Index([2912, 3817, 1559], dtype='int64')
['Baseline', 'Post Injection', 'Recovery']


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

169
8280
Int64Index([2121, 3680, 1966], dtype='int64')
['Baseline', 'Post Injection', 'Recovery']


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

54
6931
Int64Index([1480, 3078, 2131], dtype='int64')
['Baseline', 'Post Injection', 'Recovery']


...

56
9917
Int64Index([4410, 1048, 4273], dtype='int64')
['Baseline', 'Post Injection', 'Recovery']


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

69
12752
Int64Index([3597, 4375, 4396], dtype='int64')
['Baseline', 'Post Injection', 'Recovery']


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Include,Baseline_Filter_Less_Strict,Baseline_Filter_More_Strict,Experiment_Type,Anesthesia,Anesthesia_Dose,Flow_Threshold,Injection_Type,Injection_Success,Injection_Attempts,Injection1,Injection1_Stock_Conc,Injection1_Stock_Conc_Units,Injection1_Stock_Prep_Date,Injection1_Solvent,Injection1_Solvent_Percent,Injection1_Dosage,Injection1_Dosage_Units,Injection1_volume_ml,Injection2,Injection2_Stock_Conc,Injection2_Stock_Conc_Units,Injection2_Stock_Prep_Date,Injection2_Solvent,Injection2_Solvent_Percent,Injection2_Dosage,Injection2_Dosage_Units,Injection2_volume_ml,Protocol,Raw_Protocol,Notes,Notes for Nick,Conditions,Breath_Count,Ti,Te,PIF,PEF,TV,EV,RT,MV,f,EIP,EEP,Penh,EF50,RH,Tbox,Tbody,Patm,VCF,AV,Sr,n
0,221112_chamber1_ajc7482_m02wh,1,1,TL + MC,2022-11-12,<NA>,ajc7482,m,wh,2,filler,filler,filler,filler,filler,34.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,Baseline,2912.000000,115.265377,238.053214,5.763452,4.179368,0.383528,0.382436,147.560815,68.906704,180.000346,1.017277,23.235314,0.517816,1.864219,40.333103,20.900954,37.5,1010.992744,9.559931,589.955771,84.233932,1.0
0,221112_chamber1_ajc7482_m02wh,1,1,TL + MC,2022-11-12,<NA>,ajc7482,m,wh,2,filler,filler,filler,filler,filler,34.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,REC_first5,385.000000,100.480519,232.002597,6.533273,4.210961,0.411039,0.407013,141.490909,76.584416,187.568831,1.033766,23.703896,0.453792,2.036052,53.595065,20.355169,37.5,1010.000000,9.633740,80.984416,83.550649,1.0
0,221112_chamber1_ajc7482_m02wh,1,1,TL + MC,2022-11-12,<NA>,ajc7482,m,wh,2,filler,filler,filler,filler,filler,34.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,PI_first5,344.000000,91.299419,158.744186,6.677151,5.412733,0.392413,0.390262,96.209302,95.427326,244.566860,1.066860,16.659884,0.629884,2.879971,59.793605,20.498023,37.5,1011.000000,9.814012,67.880814,61.886628,1.0
0,221112_chamber1_ajc7482_m02wh,1,1,TL + MC,2022-11-12,<NA>,ajc7482,m,wh,2,filler,filler,filler,filler,filler,34.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,PI_0-10,416.000000,135.112654,265.310453,4.531307,3.663089,0.331637,0.331706,144.178674,53.330888,160.320671,0.985067,34.383809,0.763681,1.645062,55.529631,20.544627,37.5,1010.716793,9.742643,662.908567,89.088551,1.0
0,221112_chamber1_ajc7482_m02wh,1,1,TL + MC,2022-11-12,<NA>,ajc7482,m,wh,2,filler,filler,filler,filler,filler,34.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,PI_0-15,853.000000,135.112654,265.310453,4.531307,3.663089,0.331637,0.331706,144.178674,53.330888,160.320671,0.985067,34.383809,0.763681,1.645062,55.529631,20.544627,37.5,1010.716793,9.742643,662.908567,89.088551,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,221214_chamber2_ajc7485_m00wh,2,6,TL + MC,2022-12-14,<NA>,ajc7485,m,wh,0,filler,filler,filler,filler,filler,45.4,T

In [333]:
# Normalize MV, TV by weight
master_data["MV/g"] = master_data["MV"] / master_data["Weight"]
master_data["TV/g"] = master_data["TV"] / master_data["Weight"]
master_data

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Include,Baseline_Filter_Less_Strict,Baseline_Filter_More_Strict,Experiment_Type,Anesthesia,Anesthesia_Dose,Flow_Threshold,Injection_Type,Injection_Success,Injection_Attempts,Injection1,Injection1_Stock_Conc,Injection1_Stock_Conc_Units,Injection1_Stock_Prep_Date,Injection1_Solvent,Injection1_Solvent_Percent,Injection1_Dosage,Injection1_Dosage_Units,Injection1_volume_ml,Injection2,Injection2_Stock_Conc,Injection2_Stock_Conc_Units,Injection2_Stock_Prep_Date,Injection2_Solvent,Injection2_Solvent_Percent,Injection2_Dosage,Injection2_Dosage_Units,Injection2_volume_ml,Protocol,Raw_Protocol,Notes,Notes for Nick,Conditions,Breath_Count,Ti,Te,PIF,PEF,TV,EV,RT,MV,f,EIP,EEP,Penh,EF50,RH,Tbox,Tbody,Patm,VCF,AV,Sr,n,MV/g,TV/g
0,221112_chamber1_ajc7482_m02wh,1,1,TL + MC,2022-11-12,<NA>,ajc7482,m,wh,2,filler,filler,filler,filler,filler,34.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,Baseline,2912.000000,115.265377,238.053214,5.763452,4.179368,0.383528,0.382436,147.560815,68.906704,180.000346,1.017277,23.235314,0.517816,1.864219,40.333103,20.900954,37.5,1010.992744,9.559931,589.955771,84.233932,1.0,1.991523,0.011085
0,221112_chamber1_ajc7482_m02wh,1,1,TL + MC,2022-11-12,<NA>,ajc7482,m,wh,2,filler,filler,filler,filler,filler,34.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,REC_first5,385.000000,100.480519,232.002597,6.533273,4.210961,0.411039,0.407013,141.490909,76.584416,187.568831,1.033766,23.703896,0.453792,2.036052,53.595065,20.355169,37.5,1010.000000,9.633740,80.984416,83.550649,1.0,2.213422,0.011880
0,221112_chamber1_ajc7482_m02wh,1,1,TL + MC,2022-11-12,<NA>,ajc7482,m,wh,2,filler,filler,filler,filler,filler,34.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,PI_first5,344.000000,91.299419,158.744186,6.677151,5.412733,0.392413,0.390262,96.209302,95.427326,244.566860,1.066860,16.659884,0.629884,2.879971,59.793605,20.498023,37.5,1011.000000,9.814012,67.880814,61.886628,1.0,2.758015,0.011341
0,221112_chamber1_ajc7482_m02wh,1,1,TL + MC,2022-11-12,<NA>,ajc7482,m,wh,2,filler,filler,filler,filler,filler,34.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,PI_0-10,416.000000,135.112654,265.310453,4.531307,3.663089,0.331637,0.331706,144.178674,53.330888,160.320671,0.985067,34.383809,0.763681,1.645062,55.529631,20.544627,37.5,1010.716793,9.742643,662.908567,89.088551,1.0,1.541355,0.009585
0,221112_chamber1_ajc7482_m02wh,1,1,TL + MC,2022-11-12,<NA>,ajc7482,m,wh,2,filler,filler,filler,filler,filler,34.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,PI_0-15,853.000000,135.112654,265.310453,4.531307,3.663089,0.331637,0.331706,144.178674,53.330888,160.320671,0.985067,34.383809,0.763681,1.645062,55.529631,20.544627,37.5,1010.716793,9.742643,662.908567,89.088551,1.0,1.541355,0.009585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,221214

In [334]:
# Prioritize MV, TV, f columns by rearranging column order
curr_column_order = master_data.columns.tolist()
breath_count_index = curr_column_order.index("Breath_Count")
curr_column_order.remove("MV")
curr_column_order.remove("TV")
curr_column_order.remove("f")
curr_column_order.remove("MV/g")
curr_column_order.remove("TV/g")
curr_column_order.insert(breath_count_index + 1, "MV")
curr_column_order.insert(breath_count_index + 2, "TV")
curr_column_order.insert(breath_count_index + 3, "f")
curr_column_order.insert(breath_count_index + 4, "MV/g")
curr_column_order.insert(breath_count_index + 5, "TV/g")
master_data = master_data[curr_column_order]
master_data

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Include,Baseline_Filter_Less_Strict,Baseline_Filter_More_Strict,Experiment_Type,Anesthesia,Anesthesia_Dose,Flow_Threshold,Injection_Type,Injection_Success,Injection_Attempts,Injection1,Injection1_Stock_Conc,Injection1_Stock_Conc_Units,Injection1_Stock_Prep_Date,Injection1_Solvent,Injection1_Solvent_Percent,Injection1_Dosage,Injection1_Dosage_Units,Injection1_volume_ml,Injection2,Injection2_Stock_Conc,Injection2_Stock_Conc_Units,Injection2_Stock_Prep_Date,Injection2_Solvent,Injection2_Solvent_Percent,Injection2_Dosage,Injection2_Dosage_Units,Injection2_volume_ml,Protocol,Raw_Protocol,Notes,Notes for Nick,Conditions,Breath_Count,MV,TV,f,MV/g,TV/g,Ti,Te,PIF,PEF,EV,RT,EIP,EEP,Penh,EF50,RH,Tbox,Tbody,Patm,VCF,AV,Sr,n
0,221112_chamber1_ajc7482_m02wh,1,1,TL + MC,2022-11-12,<NA>,ajc7482,m,wh,2,filler,filler,filler,filler,filler,34.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,Baseline,2912.000000,68.906704,0.383528,180.000346,1.991523,0.011085,115.265377,238.053214,5.763452,4.179368,0.382436,147.560815,1.017277,23.235314,0.517816,1.864219,40.333103,20.900954,37.5,1010.992744,9.559931,589.955771,84.233932,1.0
0,221112_chamber1_ajc7482_m02wh,1,1,TL + MC,2022-11-12,<NA>,ajc7482,m,wh,2,filler,filler,filler,filler,filler,34.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,REC_first5,385.000000,76.584416,0.411039,187.568831,2.213422,0.011880,100.480519,232.002597,6.533273,4.210961,0.407013,141.490909,1.033766,23.703896,0.453792,2.036052,53.595065,20.355169,37.5,1010.000000,9.633740,80.984416,83.550649,1.0
0,221112_chamber1_ajc7482_m02wh,1,1,TL + MC,2022-11-12,<NA>,ajc7482,m,wh,2,filler,filler,filler,filler,filler,34.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,PI_first5,344.000000,95.427326,0.392413,244.566860,2.758015,0.011341,91.299419,158.744186,6.677151,5.412733,0.390262,96.209302,1.066860,16.659884,0.629884,2.879971,59.793605,20.498023,37.5,1011.000000,9.814012,67.880814,61.886628,1.0
0,221112_chamber1_ajc7482_m02wh,1,1,TL + MC,2022-11-12,<NA>,ajc7482,m,wh,2,filler,filler,filler,filler,filler,34.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,PI_0-10,416.000000,53.330888,0.331637,160.320671,1.541355,0.009585,135.112654,265.310453,4.531307,3.663089,0.331706,144.178674,0.985067,34.383809,0.763681,1.645062,55.529631,20.544627,37.5,1010.716793,9.742643,662.908567,89.088551,1.0
0,221112_chamber1_ajc7482_m02wh,1,1,TL + MC,2022-11-12,<NA>,ajc7482,m,wh,2,filler,filler,filler,filler,filler,34.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,PI_0-15,853.000000,53.330888,0.331637,160.320671,1.541355,0.009585,135.112654,265.310453,4.531307,3.663089,0.331706,144.178674,0.985067,34.383809,0.763681,1.645062,55.529631,20.544627,37.5,1010.716793,9.742643,662.908567,89.088551,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,221214

In [335]:
# Normalize MV and TV columns for Hypoxia/Normoxia
# TODO: WHAT TO DO WITH THIS???
# normalized = pd.DataFrame()
# normalized["MV/g"] = (master_data[master_data["Conditions"] == "Hypoxia"]["MV/g"] - master_data[master_data["Conditions"] == "Normoxia"]["MV/g"]) / master_data[master_data["Conditions"] == "Hypoxia"]["MV"]
# normalized["TV/g"] = (master_data[master_data["Conditions"] == "Hypoxia"]["TV/g"] - master_data[master_data["Conditions"] == "Normoxia"]["TV/g"]) / master_data[master_data["Conditions"] == "Hypoxia"]["TV"]
# normalized

In [336]:
# Make excel sheet easier to read by sorting by Mouse_ID and then Test_Number
master_data = master_data.sort_values(by=['Mouse_ID', 'Test_Number'])
master_data

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Include,Baseline_Filter_Less_Strict,Baseline_Filter_More_Strict,Experiment_Type,Anesthesia,Anesthesia_Dose,Flow_Threshold,Injection_Type,Injection_Success,Injection_Attempts,Injection1,Injection1_Stock_Conc,Injection1_Stock_Conc_Units,Injection1_Stock_Prep_Date,Injection1_Solvent,Injection1_Solvent_Percent,Injection1_Dosage,Injection1_Dosage_Units,Injection1_volume_ml,Injection2,Injection2_Stock_Conc,Injection2_Stock_Conc_Units,Injection2_Stock_Prep_Date,Injection2_Solvent,Injection2_Solvent_Percent,Injection2_Dosage,Injection2_Dosage_Units,Injection2_volume_ml,Protocol,Raw_Protocol,Notes,Notes for Nick,Conditions,Breath_Count,MV,TV,f,MV/g,TV/g,Ti,Te,PIF,PEF,EV,RT,EIP,EEP,Penh,EF50,RH,Tbox,Tbody,Patm,VCF,AV,Sr,n
0,221113_chamber1_ajc7478_m01wh,1,1,DA + MC,2022-11-13,<NA>,ajc7478,m,wh,1,filler,filler,filler,filler,filler,40.1,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,Baseline,691.000000,115.767857,0.493125,234.340774,2.886979,0.012297,85.431548,181.476190,8.926176,6.288110,0.491384,109.785714,1.273810,18.482143,0.558705,3.404777,56.375893,20.089851,37.5,1013.0,9.610774,185.758929,50.988095,1.0
0,221113_chamber1_ajc7478_m01wh,1,1,DA + MC,2022-11-13,<NA>,ajc7478,m,wh,1,filler,filler,filler,filler,filler,40.1,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,REC_first5,351.000000,117.002849,0.420028,280.376068,2.917777,0.010475,83.572650,132.125356,7.449516,6.227721,0.419829,75.920228,1.564103,15.601140,0.679202,3.801738,50.833903,20.834843,37.5,1013.0,9.760028,73.492877,64.316239,1.0
0,221113_chamber1_ajc7478_m01wh,1,1,DA + MC,2022-11-13,<NA>,ajc7478,m,wh,1,filler,filler,filler,filler,filler,40.1,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,PI_first5,259.000000,135.115830,0.514517,262.857143,3.369472,0.012831,80.625483,150.694981,9.146757,5.658571,0.494865,92.613900,1.355212,16.714286,0.413514,3.700849,56.064093,20.197915,37.5,1013.0,9.639923,67.637066,66.625483,1.0
0,221113_chamber1_ajc7478_m01wh,1,1,DA + MC,2022-11-13,<NA>,ajc7478,m,wh,1,filler,filler,filler,filler,filler,40.1,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,PI_0-10,447.000000,97.036711,0.460858,212.258014,2.419868,0.011493,89.619442,205.910548,8.044105,5.230398,0.456996,119.249741,1.327818,24.101861,0.545305,2.807544,49.390072,20.631458,37.5,1013.0,9.658418,461.914168,60.966908,1.0
0,221113_chamber1_ajc7478_m01wh,1,1,DA + MC,2022-11-13,<NA>,ajc7478,m,wh,1,filler,filler,filler,filler,filler,40.1,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,Saline,<NA>,<NA>,<NA>,<NA>,<NA>,0,mg/kg,<NA>,"Baseline, Post Injection, Recovery","Baseline, Post Injection, Recovery",<NA>,<NA>,PI_0-15,653.000000,97.036711,0.460858,212.258014,2.419868,0.011493,89.619442,205.910548,8.044105,5.230398,0.456996,119.249741,1.327818,24.101861,0.545305,2.807544,49.390072,20.631458,37.5,1013.0,9.658418,461.914168,60.966908,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,221214_chamber2_ajc7485_m00wh,2,6,

In [337]:
# Get the averages of all protocols for each condition and test_number
mean_of_conditions_test_number = master_data.groupby(["Conditions", "Test_Number"]).agg(np.mean).reset_index()
master_data = master_data.append(mean_of_conditions_test_number)

In [338]:
# Fuunction to highlight every Test_Number == 1
def highlight_every_number_1(s):
    df = s.copy()
    #set by condition
    mask = df['Test_Number'] == "1"
    df.loc[mask, :] = 'background-color: LightGreen'
    df.loc[~mask,:] = 'color: SlateGrey'
    return df   

In [339]:
# Separate master_data based on conditions and write all to one excel sheet
condition_list = list(set(master_data["Conditions"].values))
condition_list = [str(condition) for condition in condition_list] # In case of non-string values
condition_list.sort()
writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')
for condition in condition_list:
    if condition == '' or pd.isnull(condition):
        continue
    condition_dataframe = master_data[master_data["Conditions"] == condition]
    condition_dataframe = condition_dataframe.reset_index(drop=True)
    condition_dataframe.style.apply(highlight_every_number_1, axis=None).to_excel(writer, sheet_name=str(condition))
writer.close()
print("All Done")

All Done
